<a href="https://colab.research.google.com/github/y-hiroki-radiotech/llm-final-task/blob/main/inference_vllm_example_selecter_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-community langchain-huggingface vllm triton fugashi unidic-lite wandb weave langchain-huggingface langchain-chroma datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

### プロンプトの作成部分

In [1]:
!git clone https://github.com/y-hiroki-radiotech/llm-final-task.git

fatal: destination path 'llm-final-task' already exists and is not an empty directory.


In [2]:
%cd llm-final-task

/content/llm-final-task


In [3]:
from prompt import PromptStock

prompt_stock = PromptStock()

### jsonファイルの読み込み

In [4]:
import pandas as pd

# JSONLファイルを読み込む
file_path = 'elyza-tasks-100-TV_0.jsonl'
data = pd.read_json(file_path, lines=True)

# DataFrameの表示
data

,task_id,input
0,0,野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。
1,1,花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されていま...
2,2,駅について学ぶ上で知っておくべきポイントは？
3,3,以下の説明を読んだユーザーが感じるであろう疑問は？\n\n水の都ベネチアでは、知られざるエピ...
4,4,観光に最適な列車旅を5つ挙げてください。それぞれの提案には、列車の名前、運行区間、特徴を詳し...
...,...,...
95,95,以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈します...
96,96,若くして看護師を目指し、緩和ケアの道に進む人たちは患者さんとの向き合い方や感情のコントロール...
97,97,漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。\n\n1....
98,98,次の物語の展開を予想してみましょう。\n\nまる、さんかく、しかくの顔をした3きょうだいがい...


### 新しいデータに対して、タスクのラベルを与える。

In [5]:
from transformers import pipeline

model_name = "hiroki-rad/bert-base-classification-ft"
classify_pipe = pipeline(model=model_name, device="cuda:0")

results: list[dict[str, float | str]] = []
for example in data.itertuples():
    # モデルの予測結果を取得
    model_prediction = classify_pipe(example.input)[0]
    # 正解のラベルIDをラベル名に変換
    results.append( model_prediction["label"])

data["label"] = results

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### モデルのインストール

In [5]:
from langchain_community.llms import VLLM

model_name = "hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-3000"

llm = VLLM(model=model_name,
           quantization="awq")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

INFO 12-14 01:34:01 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-14 01:34:01 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-3000', speculative_config=None, tokenizer='hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-3000', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 12-14 01:36:49 model_runner.py:1077] Loading model weights took 25.5730 GB
INFO 12-14 01:36:50 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=26.06GiB peak_torch_memory=26.52GiB memory_usage_post_profile=26.10GiB non_torch_memory=0.52GiB kv_cache_size=8.57GiB gpu_memory_utilization=0.90
INFO 12-14 01:36:50 gpu_executor.py:113] # GPU blocks: 701, # CPU blocks: 327
INFO 12-14 01:36:50 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 2.74x
INFO 12-14 01:36:53 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-14 01:36:53 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

## SemanticSimilaritExampleSelectorを実装

In [6]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

embed_model_name = "intfloat/multilingual-e5-large"
embeddings = HuggingFaceEmbeddings(model_name=embed_model_name, model_kwargs={'device': 'cuda'})

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset

df = load_dataset("elyza/ELYZA-tasks-100", split="test")
df = df.to_pandas()

examples = []
for row in df.itertuples():
    examples.append({"input": row.input, "output": row.output})

README.md:   0%|          | 0.00/8.37k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="質問: {input}\n回答: {output}",
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="以下に質問と回答の具体例を提示します。",
    suffix="以上の例を参考にして、質問をよく読んで、適切な回答を簡潔に出力してください。\n質問: {input}\n回答:",
    input_variables=["input"],
    example_separator="\n\n",
    template_format="jinja2"
)

In [9]:
vllm_chain = few_shot_prompt | llm

In [10]:
import weave

weave.init("llm-8b-3000-datasets-train-few-shot-2")

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: y-hiroki-rad.
View Weave data at https://wandb.ai/y-hiroki-rad/llm-8b-3000-datasets-train-few-shot-2/weave


In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from tqdm import tqdm

chain = (
    RunnablePassthrough()
    | vllm_chain
    | StrOutputParser()
)

@weave.op()
def record_results(data):
    # Process each row with fixed position progress bar
    outputs = []
    total_rows = len(data)
    with tqdm(total=total_rows,
            desc="Processing rows",
            position=0,  # 固定位置を指定
            leave=True   # 処理完了後もバーを残す
            ) as pbar:
        for row in data.itertuples():
            # Invoke the chain
            output = chain.invoke({"input": row.input})
            outputs.append(output)

            # Update progress bar
            pbar.update(1)

    return outputs

In [12]:
outputs = record_results(data)

Processing rows: 100%|██████████| 100/100 [06:58<00:00,  4.18s/it]


🍩 https://wandb.ai/y-hiroki-rad/llm-8b-3000-datasets-train-few-shot-2/r/call/0193c2d0-ee8f-7730-a751-3b92896cf05e


## 出力部分の作成

In [13]:
import json

# Assuming 'data' and 'outputs' are defined from the previous code

# Create a list to store the JSONL objects
jsonl_data = []

# Iterate through the data and outputs
for i in range(len(data)):
    task_id = data.iloc[i]["task_id"] # Access task_id using the index
    output = outputs[i]

    # Create a dictionary for each row
    jsonl_object = {
        "task_id": task_id,
        "output": output
    }
    jsonl_data.append(jsonl_object)

with open("output.jsonl", "w", encoding="utf-8") as outfile:
  for entry in jsonl_data:
    # Convert task_id to a regular Python integer before dumping
    entry["task_id"] = int(entry["task_id"])
    json.dump(entry, outfile, ensure_ascii=False)
    outfile.write('\n')